In [1]:
import math
import numpy as np
import pandas as pd

import pickle

dtor = math.pi/180

# Build GW contours

# suggest adding this GW contour maker code to the GCN listener, as it takes O(10-20 s) to run, and so should only be run once and then stored in the same directory as the skymapfile

In [6]:
# (optional, only if you want to recreate contours)
# (otherwise, skip down to loading contours from file

from astropy.utils.data import download_file
import ligo.skymap.io
import ligo.skymap
import time

#url= 'https://gracedb.ligo.org/apiweb/superevents/S190510g/files/LALInference.fits.gz,0'
# url = 'https://gracedb.ligo.org/apiweb/superevents/S190425z/files/LALInference.fits.gz,0'
# skymapfile = download_file(url, cache=True) 

# skymap, metadata = ligo.skymap.io.fits.read_sky_map(url, nest=None)

# VERY VERY VERY SLOW. Only run once to generate the file. There is better way to do this. 
# Writes to .geojson format file.
# from ligo.skymap.tool.ligo_skymap_contour import main

# now = time.time()
# main([skymapfile,'--contour','50','90','--simplify', '-o', 'contours-smooth.json']) #add --simplify
# end = time.time()
# print(end-now)

20.251483917236328


# Now load the contours from where you saved them

In [7]:
#smoothed caled contours.json in home directory
contours_data=pd.read_json('contours-smooth.json')

In [8]:
pathlist = []
for contour in contours_data['features']:
    level = contour['properties']['credible_level']
    pathlist.extend(contour['geometry']['coordinates'])

# Fetch tiles from treasureMap

In [9]:
BASE = 'http://treasuremap.space/api/v0/'
TARGET = 'pointings'

In [10]:
import requests
payload = {'graceid':'S190425z'}
r = requests.get(url = BASE+TARGET, params = payload)

In [11]:
r.url

'http://treasuremap.space/api/v0/pointings?graceid=S190425z'

In [12]:
results = r.json()

In [13]:
pointings = []
for tile in results:
    tile = eval(tile)
    position = tile["position"]
    ra = float(position.split()[1].split('(')[1])
    dec = float(position.split()[2].split(')')[0])
    pos_angle = tile['pos_angle']
    instrument_id = tile['instrumentid']
    pointings.append([instrument_id, ra,dec,pos_angle])


# Fetch footprints dictionary

In [14]:
TARGET='footprints'
r = requests.get(url = BASE+TARGET)

In [15]:
results = r.json()

In [16]:
footprints_dict={}
for footprint_obj in results:
    footprint_obj = eval(footprint_obj)
    inst_id = int(footprint_obj['instrumentid'])
    footprint = footprint_obj['footprint']
    sanitized = footprint.strip('POLYGON ').strip(')(').split(',')
    footprint = []
    for vertex in sanitized:
        obj = vertex.split()
        ra = float(obj[0])
        dec = float(obj[1])
        footprint.append([ra,dec])
    new_entry = {inst_id:footprint}
    footprints_dict.update(new_entry)

In [17]:
footprints_dict

{9: [[-0.22083, -0.22083],
  [-0.22083, 0.22083],
  [0.22083, 0.22083],
  [0.22083, -0.22083],
  [-0.22083, -0.22083]],
 10: [[-0.08333, -0.08333],
  [-0.08333, 0.08333],
  [0.08333, 0.08333],
  [0.08333, -0.08333],
  [-0.08333, -0.08333]],
 11: [[-1.118, -1.118],
  [-1.118, 1.118],
  [1.118, 1.118],
  [1.118, -1.118],
  [-1.118, -1.118]],
 12: [[-0.141667, -0.141667],
  [-0.141667, 0.141667],
  [0.141667, 0.141667],
  [0.141667, -0.141667],
  [-0.141667, -0.141667]],
 13: [[0.2, 0.0],
  [0.1827090915285202, -0.08134732861516003],
  [0.1338261212717717, -0.1486289650954788],
  [0.06180339887498949, -0.1902113032590307],
  [-0.02090569265353067, -0.1989043790736547],
  [-0.09999999999999996, -0.1732050807568878],
  [-0.1618033988749895, -0.1175570504584947],
  [-0.1956295201467611, -0.04158233816355195],
  [-0.1956295201467612, 0.04158233816355181],
  [-0.1618033988749895, 0.1175570504584946],
  [-0.1000000000000001, 0.1732050807568877],
  [-0.02090569265353085, 0.1989043790736547],
  [

# Fetch Instruments table

In [18]:
TARGET='instruments'
r = requests.get(url = BASE+TARGET)

In [19]:
results=r.json()

In [20]:
instrument_dict={}
for instrument in results:
    instrument=eval(instrument)
    inst_id = instrument['id']
    inst_name = instrument['instrument_name']
    new_entry = {inst_id: inst_name}
    instrument_dict.update(new_entry)

In [21]:
instrument_dict

{9: 'Sinistro',
 10: 'DLT40',
 11: 'MLS10KCCD-CSS',
 12: 'Swift_UVOT',
 13: 'Swift_XRT',
 22: 'MMTCam',
 23: 'MMIRS',
 24: 'MMIRS_1024',
 25: 'Binospec',
 26: 'Binospec_1024',
 38: 'DECam',
 39: 'Mont4k'}

# Define rotation and reproject functions

In [22]:
def rotate(footprint, angle):
        #footprint is a list of vertices points
        #   vertice points = [ra, dec]

        # angle is the pos_angle of the pointing

        rot_footprint = []
        for p in footprint:
                x, y = p[0], p[1]
                r = math.sqrt(x*x + y*y)
                if x < 0:
                        r = (-1.0)*r
                theta = math.atan(y/x)-angle
                new_x = r*math.cos(theta)
                new_y = r*math.sin(theta)
                rot_footprint.append([new_x, new_y])

        return rot_footprint

In [23]:
def project(footprint, _ra, _dec):
        #footprint is a list of vertices points
        #   vertice points = [ra, dec]

        #_ra and _dec are the translated coordinates

        proj_footprint = []
        for p in footprint:
            if p[0]+_ra > 360:
                  ra = 360 - p[0]+_ra
            elif p[0]+_ra < 0:
                  ra = 360 + p[0]+_ra
            else:
                  ra = p[0] + _ra

            if p[1]+_dec > 90:
                dec = 90 - p[1]+_dec
            elif p[1]+_dec < -90:
                dec = -90 + p[1] + _dec
#             elif p[1]+_dec < 0:
#                   dec = 90 -  p[1]+_dec
            else:
                  dec = p[1] + _dec

            proj_footprint.append([ra, dec])
        return proj_footprint

# Functions for building the JS objects

In [24]:
#this takes a polygon in skycoords in the form of a list of tuples of ra,decs ie ((0,0)(0,1)(1,1)(1,0))
#and returns a footprint object
def polygons2footprints(polygons):
    #split list of poly corners into ra list and dec list
    footprints=""
    first = True
    for polygon in polygons:
        if first == True:
            first = False
        else:
            footprints += ','
        if type(polygon[0]) != list:
            polygon = [l.tolist() for l in polygon]
    
        footprint = f"A.polygon({polygon})"
        footprints += footprint
    return footprints

In [25]:
def createOverlayObject(instrumentname,footprints,color):
    javasafe_name = ''.join(e for e in instrumentname if e.isalnum())
    commandstring = f"var {javasafe_name} = A.graphicOverlay({{color: '{color}', lineWidth: 2, name:'{instrumentname}'}});\n"
    commandstring += f"aladin.addOverlay({javasafe_name});\n"
    commandstring += f"{javasafe_name}.addFootprints([{footprints}]);"
    
    return commandstring

# Get cracking with the building the list of actual footprints

In [26]:
final_foots=[]
for point in pointings:
    instrument_id = point[0]
    ra = point[1]
    dec= point[2]
    pos_angle = point[3]
    
    footprint = footprints_dict[instrument_id]
    #    name = instrument_dict[instrument_id]

    rotated_foot = rotate(footprint, pos_angle)
    projected_foot = project(rotated_foot, ra,dec) #reproject to RA, Dec of pointing
    #final_foot = rotate(projected_foot, pos_angle) #rotate in place accord to positition angle
    
    final_foots.append([instrument_id, projected_foot])

In [27]:
final_foots.sort() #sort by instrument ID

In [28]:
from itertools import *
from operator import itemgetter, attrgetter

groups = groupby(final_foots, key=itemgetter(0))

In [29]:
result = []
for key,valuesiter in groupby(final_foots, key=itemgetter(0)):
    result.append(dict(type=key, items=list(v[1] for v in valuesiter)))

In [32]:
colorlist=['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#42d4f4', '#f032e6', '#fabebe', '#469990', '#e6beff', '#9A6324', '#fffac8', '#800000', '#aaffc3', '#000075', '#a9a9a9']

In [33]:
color = colorlist[0]
contours = polygons2footprints(pathlist)
overlayObject = createOverlayObject('GW Contours',contours,color)
i = 1
payload = f"{overlayObject}\n"
for element in result:
    instrument_num = element['type']
    polylist = element['items']
    footprints = polygons2footprints(polylist)
    color=colorlist[i]
    instrumentname = instrument_dict[instrument_num]
    overlayObject = createOverlayObject(instrumentname,footprints,color)
    payload += f"{overlayObject}\n"
    i+=1


In [34]:
html = """
<!DOCTYPE>
<html>
  <body>
    <h1>Swift/UVOT follow-up of S190510g</h1>
<link rel="stylesheet" href="http://aladin.u-strasbg.fr/AladinLite/api/v2/latest/aladin.min.css" />
<script type="text/javascript" src="http://code.jquery.com/jquery-1.9.1.min.js" charset="utf-8"></script>
<div id="aladin-lite-div" style="width:900px;height:500px;"></div>
<script type="text/javascript" src="http://aladin.u-strasbg.fr/AladinLite/api/v2/latest/aladin.min.js" charset="utf-8"></script>

<!-- Creation of Aladin Lite instance with initial parameters -->
<script type="text/javascript">
    var aladin = A.aladin('#aladin-lite-div', {survey: "P/DSS2/color", fov:1.5, target: "90 -30"});
    $('input[name=survey]').change(function() {
    aladin.setImageSurvey($(this).val());});\n"""

tail = """
</script>
  </body>
</html>
"""

In [35]:
site=html+payload+tail

In [36]:
file1 = open("index.html","w") 
file1.write(site)
file1.close() 